# PostgreSQL 16.6+

## 0) 前提

* エンジン: **PostgreSQL 16.6+**
* 並び順: 任意
* `NOT IN` 回避（`EXISTS` / `LEFT JOIN ... IS NULL` を推奨）
* 判定は ID 基準、表示は仕様どおり

## 1) 問題

* `同一の tiv_2015 を持つ保険契約者が 1 名以上存在し、かつ (lat, lon) の位置が他と重複しない契約者の 2016 年投資額 (tiv_2016) を合計し、**小数 2 桁**に丸めて 1 行で出力せよ。`
* 入力: `Insurance(pid int PRIMARY KEY, tiv_2015 float, tiv_2016 float, lat float NOT NULL, lon float NOT NULL)`
* 出力: `tiv_2016`（合計、**2 桁丸め**、1 行 1 列）

## 2) 最適解（単一クエリ）

> PostgreSQL では **CTE + ウィンドウ** で明快に。必要なら `LATERAL` で「グループごとに上位 k を少量抽出」。

```sql
WITH pre AS (
  SELECT pid, tiv_2015, tiv_2016, lat, lon
  FROM Insurance
),
win AS (
  SELECT
    pid,
    tiv_2015,
    tiv_2016,
    lat,
    lon,
    COUNT(*) OVER (PARTITION BY tiv_2015) AS cnt_same_tiv2015,
    COUNT(*) OVER (PARTITION BY lat, lon) AS cnt_same_city
  FROM pre
)
SELECT
  ROUND(SUM(tiv_2016)::numeric, 2) AS tiv_2016
FROM win
WHERE cnt_same_tiv2015 > 1
  AND cnt_same_city = 1;

-- Runtime 295 ms
-- Beats 29.54%

```

* `COUNT() OVER (PARTITION BY tiv_2015)` が **2 以上** → 同額 `tiv_2015` の存在
* `COUNT() OVER (PARTITION BY lat, lon)` が **1** → `(lat,lon)` がユニーク
* 浮動小数の表示ブレを避けるため **`SUM(...)::numeric` を `ROUND(...,2)`** で丸め

### 代替（LATERAL で上位 k を作る）

> 本問題では不要だが、パターンとして掲載。

```sql
SELECT {{FINAL_PROJECTION}}
FROM {{GROUP_TABLE}} g
JOIN LATERAL (
  SELECT {{SUB_COLUMNS}}
  FROM {{DETAIL_TABLE}} d
  WHERE {{d_group_key}} = g.{{g_key}}
  {{OPTIONAL_DISTINCT_OR_GROUP}}
  ORDER BY {{SUB_ORDER}}
  LIMIT {{K}}
) s ON TRUE
{{OPTIONAL_JOINS}};
```

## 3) 要点解説

* **ウィンドウ関数で重複/一意を同時判定** → 最終行だけ合計
* 丸めは **合計→丸め**（行単位で丸めてから合算は不可）
* 型は `float` でも、**見た目精度**のため `::numeric` にキャストして `ROUND` が安全
* `EXISTS`/事前集約（半結合）でも解けるが、PostgreSQL はウィンドウ最適化が強力で実装が簡潔

## 4) 計算量（概算）

* ウィンドウ処理: **O(Σ n_g log n_g)**（内部ソートが入るケース）〜 **O(N)**（最適化ヒット時）
* インデックスで **Hash Join / Index Scan** を選べる状況なら線形近似

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 テーブル] --> B[前処理 ユニーク化や集約]
  B --> C[ウィンドウやラテラルで抽出]
  C --> D[条件で必要行を抽出]
  D --> E[必要なテーブルと結合]
  E --> F[出力 仕様列のみ]
```


# PostgreSQL 16.6+

## 0) 前提

* エンジン: **PostgreSQL 16.6+**
* 並び順: 任意
* `NOT IN` 回避（`EXISTS` / `LEFT JOIN ... IS NULL` を推奨）
* 判定は ID 基準、表示は仕様どおり

## 1) 問題

* `同一の tiv_2015 を持つ保険契約者が 1 名以上存在し、かつ (lat, lon) の位置が他と重複しない契約者の 2016 年投資額 (tiv_2016) を合計し、**小数 2 桁**に丸めて 1 行で出力せよ。`
* 入力: `Insurance(pid int PRIMARY KEY, tiv_2015 float, tiv_2016 float, lat float NOT NULL, lon float NOT NULL)`
* 出力: `tiv_2016`（合計、**2 桁丸め**、1 行 1 列）

## 2) 最適解（単一クエリ）

> **パフォーマンス重視**: 先にキーを**事前集約**して小さな集合に圧縮し、**半結合（セミジョイン）**で本体に当てる。ウィンドウよりワークが小さくなりやすく、`GROUP BY` がインデックスに乗れば高速化が見込めます。

```sql
-- ① 実行クエリ（事前集約＋半結合）
SELECT ROUND(SUM(i.tiv_2016)::numeric, 2) AS tiv_2016
FROM Insurance AS i
JOIN (
  SELECT tiv_2015
  FROM Insurance
  GROUP BY tiv_2015
  HAVING COUNT(*) > 1
) AS d USING (tiv_2015)
JOIN (
  SELECT lat, lon
  FROM Insurance
  GROUP BY lat, lon
  HAVING COUNT(*) = 1
) AS u USING (lat, lon);

-- Runtime 241 ms
-- Beats 96.00%

```

### 代替（LATERAL で上位 k を作る）

> 今回は上位抽出が不要のため **参考パターン** のみ。

```sql
SELECT {{FINAL_PROJECTION}}
FROM {{GROUP_TABLE}} g
JOIN LATERAL (
  SELECT {{SUB_COLUMNS}}
  FROM {{DETAIL_TABLE}} d
  WHERE {{d_group_key}} = g.{{g_key}}
  {{OPTIONAL_DISTINCT_OR_GROUP}}
  ORDER BY {{SUB_ORDER}}
  LIMIT {{K}}
) s ON TRUE
{{OPTIONAL_JOINS}};
```

## 3) 要点解説

**なぜ速い？**

* `d` は「**重複する tiv_2015**」だけ、`u` は「**一意の (lat,lon)**」だけを持つ**縮小集合**。
  本体 `Insurance` に直接ウィンドウ集計を当てるより **I/O とワークテーブルが減る**傾向。
* PostgreSQL は `GROUP BY` に **HashAggregate** を選びやすく、キーに索引があると `GROUP BY` と `JOIN` が効率化。

**さらに効かせる作戦（順に試す）**

1. **適切な索引（最重要）**

```sql
-- GROUP BY と USING 結合を支える基本2本
CREATE INDEX IF NOT EXISTS ix_insurance_tiv2015   ON Insurance (tiv_2015);
CREATE INDEX IF NOT EXISTS ix_insurance_lat_lon   ON Insurance (lat, lon);

-- （任意）Index Only Scan を狙う：表示列を含める（PostgreSQL 11+）
-- ただし可視性マップ次第。効果はワークロード依存。
CREATE INDEX IF NOT EXISTS ix_insurance_tiv2015_inc ON Insurance (tiv_2015) INCLUDE (tiv_2016, lat, lon);
```

2. **実行計画の確認と軽微なヒント**

```sql
EXPLAIN (ANALYZE, BUFFERS, VERBOSE)
SELECT ROUND(SUM(i.tiv_2016)::numeric, 2) AS tiv_2016
FROM Insurance AS i
JOIN (
  SELECT tiv_2015 FROM Insurance GROUP BY tiv_2015 HAVING COUNT(*) > 1
) d USING (tiv_2015)
JOIN (
  SELECT lat, lon FROM Insurance GROUP BY lat, lon HAVING COUNT(*) = 1
) u USING (lat, lon);
```

* 期待: `HashAggregate` → `Hash Join`（or `Merge Join` も可）、**Seq Scan でも OK**（表が中小規模なら問題なし）。
* `Using index for group by` は MySQL 用語。PostgreSQL では **HashAggregate** / **GroupAggregate** と **Index/Bitmap/Seq Scan** の組合せを確認。

3. **ワークメモリの一時アップ（集約・ハッシュ結合が溢れている場合のみ）**

```sql
-- セッション限定で試験（サーバ全体は DBA 方針に従う）
SET work_mem = '128MB';
-- 場合により 64–256MB 程度まで。上げすぎ注意（同時接続数×work_mem がメモリ圧に）。
```

4. **テーブル統計の最新化**

```sql
ANALYZE Insurance;  -- 統計が古いと誤プランで遅くなる
```

5. **バキューム/可視性の改善（Index Only Scan 期待時）**

* 自動 VACUUM が追いつかない環境なら `VACUUM (ANALYZE)` の運用を見直し。
* 可視性マップが進むと Index Only Scan が生きやすくなります。

6. **EXISTS 版（読みやすさ重視・索引が効けば速い）**

```sql
SELECT ROUND(SUM(i.tiv_2016)::numeric, 2) AS tiv_2016
FROM Insurance i
WHERE EXISTS (
  SELECT 1
  FROM Insurance s
  WHERE s.tiv_2015 = i.tiv_2015
    AND s.pid <> i.pid
)
AND NOT EXISTS (
  SELECT 1
  FROM Insurance c
  WHERE c.lat = i.lat
    AND c.lon = i.lon
    AND c.pid <> i.pid
);

-- Runtime 254 ms
-- Beats 71.95%

```

* これを速くする索引は **同じく** `INDEX(tiv_2015)` と `INDEX(lat,lon)`。
* 行数が多く **重複/一意キーの分布に偏り**がある場合、事前集約版がより安定して速いことが多いです。

## 4) 計算量（概算）

* **事前集約＋半結合**: `GROUP BY` 2回 + 2回の `JOIN`。ハッシュ集約/結合がメモリ内で収まればおおむね **O(N)** 近似。
* **EXISTS 版**: 索引ヒット時は各相関が平均 **O(1)** 近似で **O(N)**。ヒットしないと **O(N²)** に悪化。
* **ウィンドウ版**: 全表に対し 2 つのパーティションで `COUNT OVER` → **O(N log N)**（内部ソート）になり得る。

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[Insurance 全件] --> B[集約 d: tiv_2015 ごと COUNT>1]
  A --> C["集約 u: (lat,lon) ごと COUNT=1"]
  B --> D["半結合 USING (tiv_2015)"]
  C --> D
  D --> E["SUM(tiv_2016) → ROUND(…,2)"]
  E --> F["出力 tiv_2016(1行)"]
```

---

### クイックチェックリスト

* [ ] 上記 **事前集約＋半結合** クエリに差し替えた
* [ ] `ix_insurance_tiv2015`, `ix_insurance_lat_lon` を作成済み
* [ ] `EXPLAIN (ANALYZE, BUFFERS)` で **HashAggregate/Hash Join** がメモリ内で完結している
* [ ] 必要に応じ `work_mem` を一時的に調整
* [ ] `ANALYZE Insurance;` 済み（統計を新鮮に）

この組み合わせで、多くのケースで **ウィンドウ版より短縮**が期待できます。
